# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [1]:
import glob

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

# TODO: Update SET_NAME and SET_DIR
SET_NAME = "diurnal_cycle"
SET_DIR = "861-time-series-multiple"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/**"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/{SET_NAME}/**"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [2]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_count = 0

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(SET_DIR, "main")

        if fp_dev not in MAIN_GLOB:
            print(f"No production file found to compare with {fp_dev}!")
            missing_count += 1

    for fp_dev in DEV_GLOB:
        fp_main = fp_main.replace("main", SET_DIR)

        if fp_main not in DEV_GLOB:
            print(f"No development file found to compare with {fp_main}!")
            missing_count += 1

    print(f"Number of files missing: {missing_count}")

In [3]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    for fp_main in MAIN_GLOB:
        if "test.nc" in fp_main or "ref.nc" in fp_main:
            fp_dev = fp_main.replace("main", SET_DIR)

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)

            var_keys = [
                "PRECT_diurnal_cycmean",
                "PRECT_diurnal_amplitude",
                "PRECT_diurnal_phase",
            ]
            for key in var_keys:
                print(f"    * var_key: {key}")

                dev_data = ds1[key].values
                main_data = ds2[key].values

                if dev_data is None or main_data is None:
                    print("    * Could not find variable key in the dataset(s)")
                    continue

                try:
                    np.testing.assert_allclose(
                        dev_data,
                        main_data,
                        atol=atol,
                        rtol=rtol,
                    )
                except (KeyError, AssertionError) as e:
                    print(f"    {e}")
                else:
                    print(f"    * All close and within relative tolerance ({rtol})")

## 1. Check for matching and equal number of files


In [4]:
_check_if_files_found()

In [5]:
_check_if_missing_files()

Number of files missing: 0


In [6]:
_check_if_matching_filecount()

Matching file count (60 and 60).


## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [7]:
_get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N_ref.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N_ref.nc
    * var_key: PRECT_diurnal_cycmean
    * All close and within relative tolerance (1e-05)
    * var_key: PRECT_diurnal_amplitude
    * All close and within relative tolerance (1e-05)
    * var_key: PRECT_diurnal_phase
    * All close and within relative tolerance (1e-05)
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/861-time-series-multiple/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/diurnal_cycle/TRMM-3B43v-7_3hr/TRMM-3B43v-7_3hr-PRECT-ANN-20S20N_test.nc
    * var_key: PRECT_diurnal_cycmean
    * All close and within relative tolerance (1e-05)
    * var_key: PRECT_diurnal_amplitude
    * All clo

### Results

- All files are within rtol 1e-05.
